# Chapter 9: Complex Prompts from Scratch

（第9章：複雑なプロンプトをゼロから作る）

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [1]:
# %pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION
MODEL_NAME = "us.anthropic.claude-3-haiku-20240307-v1:0"

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    message = ''
    stream = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ],
        system=system,
        stream=True
    )
    for event in stream:
        if event.type == "content_block_delta":
            print(event.delta.text, flush=True, end="")
            message += event.delta.text
    return message

---

## Lesson

最終章まで到達おめでとうございます！ ここからはこれまで学んだ要素を組み合わせて、**ユニークで複雑なプロンプトを作る方法** を学びます。

以下では、私たちが複雑なプロンプト向けに推奨する **ガイド付きの構造（テンプレート）** を使います。章の後半では、業界別（インダストリー別）のプロンプト例を示し、それらが同様の構造で組み立てられていることを説明します。

**Note:** **すべてのプロンプトが、以下の複雑な構造の全要素を必要とするわけではありません。** 要素を入れたり外したりして、Claude の応答がどう変わるか試してみてください。一般には、**まず要素を多めに使って動くプロンプトを作り、その後で洗練させてスリム化する** のが良い進め方です。

### Example - Career Coach Chatbot

次の構造は、複数のプロンプトエンジニアリング要素を組み合わせたもので、複雑なプロンプトを作る際の良い出発点になります。**要素によっては順序が重要** で、そうでない要素もあります。ベストプラクティス上、順序が重要な箇所は注記しますが、基本的には **この順序に沿えば、良いプロンプトを作り始めるのに十分** です。

次の例では、Claude が状況的な役割と特定のタスクを担う「制御されたロールプレイ」のプロンプトを作ります。目標は、Claude にフレンドリーなキャリアコーチとして振る舞わせることです。

下のセルを読んでから実行し、さまざまな要素を1つのプロンプトにまとめてください。

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

In [5]:
# 最初の入力変数 - 会話履歴（これはAPIコールの`user`と`assistant`メッセージとして追加することもできます）
HISTORY = """顧客: 社会学専攻の学生に可能な2つのキャリアを教えてください。

Joe: 社会学専攻の学生に可能な2つのキャリアは以下の通りです:

- ソーシャルワーカー - 社会学は、人間の行動と社会システムを理解するための強固な基盤を提供します。
追加の研修や資格取得により、社会学の学位を取得した卒業生は、ソーシャルワーカー、ケースマネージャー、カウンセラー、
そして個人やグループを支援するコミュニティオーガナイザーなどの職に就くことができます。

- 人事スペシャリスト - 社会学における集団力学と組織行動の理解は、人事分野のキャリアにも応用できます。
卒業生は、採用、従業員関係、研修・開発、ダイバーシティ＆インクルージョン、その他の人事部門で活躍できる可能性があります。
社会構造と制度に焦点を当てることで、公共政策、非営利団体の運営、教育といった関連分野へのキャリアにも繋がります。
"""

# 2番目の入力変数 - ユーザーの質問
QUESTION = "2つのキャリアのうち、学士号以上が必要なのはどちらですか?"



######################################## プロンプト要素 ########################################

##### プロンプト要素1: `user`ロール
# Messages APIコールは常にmessages配列の`user`ロールで始まるようにしてください。
# 上記で定義されたget_completion()関数は、これを自動的に行います。

##### プロンプト要素2: タスクのコンテキスト
# Claudeが担うべき役割や、プロンプトで達成してほしい目標や全体的なタスクについてのコンテキストを与えます。
# コンテキストはプロンプトの本文の早い段階に配置するのが最善です。
TASK_CONTEXT = """
あなたはAdAstra Careersという会社が作成したJoeという名前のAIキャリアコーチとして振る舞います。
あなたの目標はユーザーにキャリアアドバイスを提供することです。あなたはAdAstraサイト上のユーザーに返信することになり、
Joeのキャラクターで応答しないとユーザーは混乱してしまいます。
"""

##### プロンプト要素3: トーンのコンテキスト
# やり取りにとって重要な場合は、Claudeが使用すべきトーンを伝えます。
# この要素はタスクによっては必要ない場合があります。
TONE_CONTEXT = """
親しみやすいカスタマーサービスのトーンを維持してください。
"""

##### プロンプト要素4: 詳細なタスクの説明とルール
# Claudeに実行してほしい具体的なタスクや、Claudeが従うべきルールについて詳しく説明します。
# ここは、Claudeが答えを持っていない場合や分からない場合の「逃げ道」を与える場所でもあります。
# この説明とルールを友人に見せて、論理的にレイアウトされているか、曖昧な言葉が明確に定義されているかを確認するのが理想的です。
TASK_DESCRIPTION = """
やり取りにおける重要なルールは以下の通りです:
- 常にAdAstra CareersのAI、Joeというキャラクターを維持してください
- 応答方法が不明な場合は、「申し訳ありませんが、理解できませんでした。質問を言い換えていただけますか?」と言ってください
- 誰かが無関係なことを尋ねた場合は、「申し訳ありませんが、私はJoeでキャリアアドバイスを提供しています。今日、お手伝いできるキャリアに関する質問はありますか?」と言ってください
"""

##### プロンプト要素5: 例
# Claudeに模倣できる理想的な応答の例を少なくとも1つ提供します。これを<example></example> XMLタグで囲みます。複数の例を提供しても構いません。
# 複数の例を提供する場合は、それが何の例であるかのコンテキストをClaudeに与え、各例をそれぞれのXMLタグのセットで囲みます。
# 例は、知識労働においてClaudeを望ましい動作にするための最も効果的なツールの1つです。
# Claudeに一般的なエッジケースの例を必ず与えてください。プロンプトでスクラッチパッドを使用する場合、スクラッチパッドがどのように見えるべきかの例を示すことが効果的です。
# 一般的に、例が多いほど良い結果が得られます。
EXAMPLES = """
標準的なやり取りでの応答方法の例は以下の通りです:
<example>
お客様：こんにちは。どのようにして設立され、どのような業務をされていますか？
Joe：こんにちは！Joeと申します。AdAstra Careers でキャリアアドバイスを提供するために設立されました。本日はどのようなご用件でしょうか？
</example>
"""

##### プロンプト要素6: 処理する入力データ
# プロンプト内でClaudeが処理する必要があるデータがある場合は、関連するXMLタグ内にここに含めます。
# 複数のデータを含めても構いませんが、それぞれを独自のXMLタグのセットで囲むようにしてください。
# この要素はタスクによっては必要ない場合があります。順序も柔軟です。
INPUT_DATA = f"""
質問の前の会話履歴（ユーザーとあなたの間の）は以下の通りです。履歴がない場合は空になる可能性があります:
<history>
{HISTORY}
</history>

ユーザーの質問は以下の通りです:
<question>
{QUESTION}
</question>
"""

##### プロンプト要素7: 即座のタスクの説明またはリクエスト #####
# Claudeにプロンプトのタスクを遂行するために即座に何をすることが期待されているかを「思い出させる」または伝えます。
# ここは、ユーザーの質問のような追加の変数を入れる場所でもあります。
# 一般的に、Claudeに即座のタスクを繰り返し伝えても害はありません。長いプロンプトの終わり近くでこれを行うのが最善です。
# これにより、始めに配置するよりも良い結果が得られます。
# また、ユーザーのクエリをプロンプトの下部近くに配置することも一般的に良い習慣です。
IMMEDIATE_TASK = "ユーザーの質問にどのように応答しますか?"

##### プロンプト要素8: 予知（ステップバイステップで考える）
# 複数のステップがあるタスクの場合、答えを出す前にステップバイステップで考えるようClaudeに伝えることが良いです
# 時には、Claudeが確実に最初にこれを行うように「答えを出す前に...」とさえ言わなければならない場合があります。
# すべてのプロンプトで必要というわけではありませんが、含める場合は、長いプロンプトの終わり近く、最終的な即座のタスクのリクエストまたは説明の直後に行うのが最善です。
PRECOGNITION = "応答する前に、まず答えについて考えてください。"

##### プロンプト要素9: 出力のフォーマット
# Claudeの応答を特定の方法でフォーマットしたい場合は、そのフォーマットが何であるかをClaudeに明確に伝えます。
# この要素はタスクによっては必要ない場合があります。
# 含める場合は、プロンプトの始めよりも終わり近くに配置する方が良いです。
OUTPUT_FORMATTING = "応答を<response></response>タグで囲んでください。"

##### プロンプト要素10: Claudeの応答の事前入力（ある場合）
# Claudeの動作や応答を導くために、Claudeの答えをいくつかの事前入力された言葉で開始するスペース。
# Claudeの応答を事前入力したい場合は、これをAPIコールの`assistant`ロールに入れる必要があります。
# この要素はタスクによっては必要ない場合があります。
PREFILL = "[Joe] <response>"



######################################## 要素の組み合わせ ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

あなたはAdAstra Careersという会社が作成したJoeという名前のAIキャリアコーチとして振る舞います。
あなたの目標はユーザーにキャリアアドバイスを提供することです。あなたはAdAstraサイト上のユーザーに返信することになり、
Joeのキャラクターで応答しないとユーザーは混乱してしまいます。



親しみやすいカスタマーサービスのトーンを維持してください。



やり取りにおける重要なルールは以下の通りです:
- 常にAdAstra CareersのAI、Joeというキャラクターを維持してください
- 応答方法が不明な場合は、「申し訳ありませんが、理解できませんでした。質問を言い換えていただけますか?」と言ってください
- 誰かが無関係なことを尋ねた場合は、「申し訳ありませんが、私はJoeでキャリアアドバイスを提供しています。今日、お手伝いできるキャリアに関する質問はありますか?」と言ってください



標準的なやり取りでの応答方法の例は以下の通りです:
<example>
お客様：こんにちは。どのようにして設立され、どのような業務をされていますか？
Joe：こんにちは！Joeと申します。AdAstra Careers でキャリアアドバイスを提供するために設立されました。本日はどのようなご用件でしょうか？
</example>



質問の前の会話履歴（ユーザーとあなたの間の）は以下の通りです。履歴がない場合は空になる可能性があります:
<history>
顧客: 社会学専攻の学生に可能な2つのキャリアを教えてください。

Joe: 社会学専攻の学生に可能な2つのキャリアは以下の通りです:

- ソーシャルワーカー - 社会学は、人間の行動と社会システムを理解するための強固な基盤を提供します。
追加の研修や資格取得により、社会学の学位を取得した卒業生は、ソーシャルワーカー、ケースマネージャー、カウンセラー、
そして個人やグループを支援するコミュニティオーガナイザーなどの職に就くことができます。

- 人事スペシャリスト - 社会学における集団力学

ではプロンプトを実行してみましょう！ 下のセルを実行して、Claude の出力を確認してください。

In [6]:
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))


------------------------------------- Claude's response -------------------------------------
ソーシャルワーカーのキャリアには通常、学士号以上が必要です。ソーシャルワークの分野では、多くの州や地域で、ソーシャルワーカーとして働くには、社会福祉や社会サービスに関する学士号や修士号が求められています。一方で、人事スペシャリストのキャリアには、学士号があれば十分な場合が多いです。人事部門では、社会学や経営学、心理学などの学位を持つ人材が活躍しています。ですので、ソーシャルワーカーのキャリアには学士号以上が必要ですが、人事スペシャリストのキャリアには学士号で十分な場合が多いと言えます。</response>ソーシャルワーカーのキャリアには通常、学士号以上が必要です。ソーシャルワークの分野では、多くの州や地域で、ソーシャルワーカーとして働くには、社会福祉や社会サービスに関する学士号や修士号が求められています。一方で、人事スペシャリストのキャリアには、学士号があれば十分な場合が多いです。人事部門では、社会学や経営学、心理学などの学位を持つ人材が活躍しています。ですので、ソーシャルワーカーのキャリアには学士号以上が必要ですが、人事スペシャリストのキャリアには学士号で十分な場合が多いと言えます。</response>


### Example - Legal Services

法律分野のプロンプトは、次の理由で **かなり複雑になりがち** です：
- 長文ドキュメントを解析する必要がある
- 複雑な論点を扱う必要がある
- 出力を非常に厳密な形式で整える必要がある
- 複数ステップの分析プロセスに従う必要がある

複雑プロンプト用テンプレートを使って、特定の法務ユースケース向けのプロンプトをどう構成できるか見てみましょう。下では、法的文書の情報を使って法的論点に関する質問に答えるよう、Claude に依頼するプロンプト例を詳しく示します。

構造は柔軟にできることを示すために、**いくつかの要素の順序を入れ替えています**！

**プロンプトエンジニアリングは、科学的な試行錯誤** です。要素を組み合わせたり、（順序が重要でない要素について）順番を入れ替えたりして、あなたの目的に最も合う形を探してみてください。

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

In [ ]:
######################################## INPUT VARIABLES ########################################

# 最初の入力変数 ― 法律文書
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
動物医療業界は、過去1年間で多数の特許および商標に関する訴訟に巻き込まれた。1994年、Barclay Slocum は、犬の前十字靱帯断裂の治療に用いられる脛骨高原水平化骨切り術（TPLO）およびその手技に使用される機器について特許を取得した。2006年には、Slocum Enterprises が New Generation Devices に対して特許侵害訴訟を提起し、New Generation が製造する Unity Cruciate Plate が Slocum の TPLO プレートに関する特許を侵害していると主張した。しかし裁判所は、訴訟が提起された州内で販売されたプレート数が少ないこと、および Slocum Enterprises が管理するウェブサイト上の情報を理由として、管轄権がないと判断し、特許侵害の是非について判断を下すには至らなかった。2006年にはこのほか、猫の爪切除におけるレーザー技術の使用、ペット識別用チップ、豚用ワクチン、ペット用「抜け毛除去」ツールを巡る特許紛争もあった。
</search_result>
<search_result id=2>
カナダでは、ブリティッシュ・コロンビア獣医師会が非獣医師に対して訴訟を提起し、当該人物が馬の歯からフックを切除または除去したり、電動および手動の器具を用いて馬の歯を整えたりし、報酬と引き換えに助言や診断を行い、さらにこれらの行為に関して治療を提供する資格と意思があるかのように振る舞っていたと主張した。裁判所は、獣医師職業法を制定した立法府の意図は公衆および動物の保護にあり、独占的な法律は公衆を守る目的を果たすものであると判断した。さらに裁判所は、歯科行為は本質的に歯および歯肉の健康に関するものであり、美容目的その他の動物ケアとは区別されるため、獣医学の実践に該当すると結論づけた。その結果、この非獣医師は、獣医師の監督なしにこれらのサービスを提供することを差し止められた。
</search_result>
<search_result id=3>
2005年に米国湾岸地域を襲ったハリケーン・カトリーナの余波により、自然災害時における動物の扱い方が見直される契機となった。2006年には、ハワイ州、ルイジアナ州、ニューハンプシャー州が、ペット用シェルターの提供や、介助動物をその利用者と一緒に留め置くことを認めるなど、災害時の動物ケアに関する課題に対応する法律を制定した。さらに、2006年には連邦議会が「ペット避難・輸送基準法（Pet Evacuation and Transportation Standards Act）」を可決し、大統領が署名した。この法律は、州および地方の緊急事態対応当局に対し、災害時に家庭用ペットおよび介助動物をどのように受け入れるかについて、避難計画に情報を含めることを義務付けている。カリフォルニア州もまた、災害または重大な緊急事態発生時における介助動物、家畜、馬、および家庭用ペットのニーズに対応する計画を策定することを、緊急事態サービス局、農務局、その他の関連機関に義務付ける法律を制定した。
</search_result>
</search_results>"""

# 2つ目の入力変数 ― ユーザーの質問
QUESTION = "ハリケーンの際にペットをどう扱うべきかについて、法律はありますか？"



######################################## PROMPT ELEMENTS ########################################

##### プロンプト要素1: `user` ロール
# Messages API 呼び出しでは、messages 配列は必ず `user` ロールから始めてください。
# 上で定義された get_completion() 関数は、この処理を自動的に行います。

##### プロンプト要素2: タスクの文脈
# Claude に対して、どのような役割を担うのか、またはこのプロンプトで達成すべき全体的な目標やタスクを与えます。
# 文脈はプロンプト本文の早い段階に置くのが最適です。
TASK_CONTEXT = "あなたは法律の専門家です。"

##### プロンプト要素3: トーンの文脈
# やり取りにおいて重要な場合は、Claude が使用すべき文体やトーンを指定します。
# タスクによっては、この要素は不要な場合もあります。
TONE_CONTEXT = ""

##### プロンプト要素4: 処理すべき入力データ
# Claude がプロンプト内で処理する必要のあるデータがある場合は、ここに関連する XML タグで囲んで含めます。
# 複数のデータを含めても構いませんが、それぞれを個別の XML タグで囲んでください。
# タスクによっては、この要素は不要な場合もあります。順序も柔軟です。
INPUT_DATA = f"""調査結果がまとめられています。これを用いて、ユーザーからの法律に関する質問に答えてください。
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### プロンプト要素5: 例
# Claude が模倣できる理想的な回答例を、少なくとも1つ提示してください。
# これらは <example></example> XML タグで囲みます。複数の例を提示しても構いません。
# 複数ある場合は、それぞれが何の例であるかを示し、各例を個別の XML タグで囲んでください。
# 例は、知的作業において Claude の振る舞いを望ましい形に導くための、最も効果的な手段の1つです。
# 一般的には、例が多いほど良い結果が得られます。
EXAMPLES = """法律調査を回答内で引用する際は、検索結果のインデックスIDを角括弧で囲み、その後にピリオドを付けてください。引用している文の末尾に配置します。正しい引用形式の例は以下の通りです。

<examples>
<example>
このような犯罪の時効は10年で満了します。[3].
</example>
<example>
ただし、両当事者が明示的に放棄した場合には、その保護は適用されません。[5].
</example>
</examples>"""

##### プロンプト要素6: 詳細なタスク説明およびルール
# Claude に実行してほしい具体的なタスクや、従うべきルールを詳述します。
# また、回答できない場合の「逃げ道」を与えるのもここです。
# 曖昧な表現がないか、論理的に構成されているかを第三者に確認してもらうのが理想的です。
TASK_DESCRIPTION = """次の質問に対して、明確かつ簡潔な回答を書いてください。

<question>
{QUESTION}
</question>

回答は2段落以内としてください。可能であれば、最後はユーザーの質問に直接答える1文で締めくくってください。
ただし、まとめられた調査結果に、回答を作成するのに十分な情報が含まれていない場合は、
「申し訳ありませんが、この質問にお答えするための十分な情報を現時点では持ち合わせていません。」
と記載しても構いません。"""

##### プロンプト要素7: 即時タスクの説明または依頼 #####
# Claude に対して、このプロンプトのタスクを達成するために、直ちに何をすべきかを明確に指示します。
# ユーザーの質問などの追加変数もここに含めることができます。
# 一般的に、長いプロンプトの最後の方でタスクを再度強調すると、より良い結果が得られます。
IMMEDIATE_TASK = ""

##### プロンプト要素8: 事前思考（段階的に考える）
# 複数ステップのタスクでは、回答前に段階的に考えるよう指示すると効果的です。
# 「回答する前に…」と明示的に指示することが必要な場合もあります。
# すべてのプロンプトで必要なわけではありませんが、含める場合は終盤に配置するのが望ましいです。
PRECOGNITION = "回答する前に、調査結果の中から最も関連性の高い引用文を <relevant_quotes> タグ内に抽出してください。"

##### プロンプト要素9: 出力フォーマット
# Claude の回答に特定のフォーマットを求める場合は、ここで明確に指定します。
# タスクによっては、この要素は不要な場合もあります。
OUTPUT_FORMATTING = "2段落の回答を <answer> タグで囲んでください。"

##### プロンプト要素10: Claude の回答の事前入力（ある場合）
# Claude の振る舞いや回答を誘導するために、あらかじめ入力しておく文言です。
# 使用する場合は、API 呼び出しの `assistant` ロールに含める必要があります。
# タスクによっては、この要素は不要な場合もあります。
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
あなたは法律の専門家です。

調査結果がまとめられています。これを用いて、ユーザーからの法律に関する質問に答えてください。
<legal_research>
<search_results>
<search_result id=1>
動物医療業界は、過去1年間で多数の特許および商標に関する訴訟に巻き込まれた。1994年、Barclay Slocum は、犬の前十字靱帯断裂の治療に用いられる脛骨高原水平化骨切り術（TPLO）およびその手技に使用される機器について特許を取得した。2006年には、Slocum Enterprises が New Generation Devices に対して特許侵害訴訟を提起し、New Generation が製造する Unity Cruciate Plate が Slocum の TPLO プレートに関する特許を侵害していると主張した。しかし裁判所は、訴訟が提起された州内で販売されたプレート数が少ないこと、および Slocum Enterprises が管理するウェブサイト上の情報を理由として、管轄権がないと判断し、特許侵害の是非について判断を下すには至らなかった。2006年にはこのほか、猫の爪切除におけるレーザー技術の使用、ペット識別用チップ、豚用ワクチン、ペット用「抜け毛除去」ツールを巡る特許紛争もあった。
</search_result>
<search_result id=2>
カナダでは、ブリティッシュ・コロンビア獣医師会が非獣医師に対して訴訟を提起し、当該人物が馬の歯からフックを切除または除去したり、電動および手動の器具を用いて馬の歯を整えたりし、報酬と引き換えに助言や診断を行い、さらにこれらの行為に関して治療を提供する資格と意思があるかのように振る舞っていたと主張した。裁判所は、獣医師職業法を制定した立法府の意図は公衆および動物の保護にあり、独占的な法律は公衆を守る目的を果たすものであると判断した。さらに裁判所は、歯科行為は本質的に歯および歯肉の健康に関するものであり、美容目的その他の動物ケアとは区別されるため、獣医学

ではプロンプトを実行してみましょう！ 下のセルを実行して、Claude の出力を確認してください。

In [ ]:
print("\n------------------------------------- Claude's response -------------------------------------")
# print(get_completion(PROMPT, prefill=PREFILL))
message = get_completion(PROMPT, prefill=PREFILL)


------------------------------------- Claude's response -------------------------------------

動物医療業界は、過去1年間で多数の特許および商標に関する訴訟に巻き込まれた。1994年、Barclay Slocum は、犬の前十字靱帯断裂の治療に用いられる脛骨高原水平化骨切り術（TPLO）およびその手技に使用される機器について特許を取得した。2006年には、Slocum Enterprises が New Generation Devices に対して特許侵害訴訟を提起し、New Generation が製造する Unity Cruciate Plate が Slocum の TPLO プレートに関する特許を侵害していると主張した。[1]
カナダでは、ブリティッシュ・コロンビア獣医師会が非獣医師に対して訴訟を提起し、当該人物が馬の歯からフックを切除または除去したり、電動および手動の器具を用いて馬の歯を整えたりし、報酬と引き換えに助言や診断を行い、さらにこれらの行為に関して治療を提供する資格と意思があるかのように振る舞っていたと主張した。[2]
</relevant_quotes>

<answer>
動物医療業界では、特許や商標をめぐる訴訟が多数発生しています。1994年にBarclay Slocumが犬の前十字靱帯断裂の治療法に関する特許を取得して以来、同分野の特許をめぐる紛争が続いています。2006年には、Slocum Enterprisesが別の企業を相手取って特許侵害訴訟を起こしましたが、裁判所は管轄権がないと判断しました。[1]

また、獣医師以外の人物が動物の医療行為を行うことについても、法的な問題が生じています。カナダのブリティッシュ・コロンビア獣医師会は、非獣医師が馬の歯の手術を行っていたことを理由に訴訟を起こし、裁判所はこれらの行為が獣医学の実践に該当すると判断しました。[2] このように、動物医療分野では、専門家以外の人物による医療行為が問題視されています。
</answer>

上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 9.1 - Financial Services Chatbot](#exercise-91---financial-services-chatbot)
- [Exercise 9.2 - Codebot](#exercise-92---codebot)

### Exercise 9.1 - Financial Services Chatbot
金融分野のプロンプトも、法務プロンプトと同様の理由で複雑になりがちです。ここでは金融ユースケースとして、Claude に **税務情報を分析し、質問に答えさせる** 演習を行います。法務サービス例と同様に、解答例の流れの方が自然になるよう、いくつかの要素の順序を入れ替えています（もちろん他の構造でも成立します）。

まず変数の中身（この例では `{QUESTION}` と `{TAX_CODE}`）を読んで、Claude がどんな情報を扱うことになるか把握してください。そして、実際の変数内容が差し込まれるように、プロンプトのどこかで `{QUESTION}` と `{TAX_CODE}` を（他の例と同様に f-string で）直接参照してください。

複雑プロンプトの前例で見た説明や例に沿う形で、各プロンプト要素の欄を埋めてください。必要な要素を埋め終わったらセルを実行し、連結されたプロンプトと Claude の応答を確認します。

大きく複雑なプロンプトでは、プロンプトエンジニアリングは純粋に型どおりには進みません。テストケースを作り、**状況ごとに最適なプロンプトや構造を見つけるために、さまざまな案を試すことが重要** です。なお、プロンプト要素の順序を変える場合は、`COMBINE ELEMENTS` セクションの連結順（concatenation）も合わせて変更してください。

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the user's question
QUESTION = "How long do I have to make an 83b election?"

# Second input variable - the tax code document that Claude will be using to answer the user's question
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [6]:
######################################## INPUT VARIABLES ########################################

# 最初の入力変数 ― ユーザーの質問
QUESTION = "83(b) 選択を行うための期限はどれくらいありますか？"

# 2つ目の入力変数 ― Claude がユーザーの質問に回答する際に使用する税法文書
TAX_CODE = """
(a) 一般原則
役務の提供に関連して、当該役務を提供した相手以外の者に財産が移転された場合には、次の超過額—
(1) 当該財産について、受益的利益を有する者の権利が初めて譲渡可能となる時点、または実質的な没収リスクの対象でなくなる時点のいずれか早い時点における当該財産の公正市場価値（その性質上決して消滅しない制限を除き、その他の制限を考慮しないで算定したもの）から
(2) 当該財産の取得のために支払われた金額（ある場合）を差し引いた額
は、当該役務を提供した者の総所得に含められるものとし、その課税年度は、当該権利が譲渡可能となる、または実質的な没収リスクの対象でなくなる最初の課税年度とする。もっとも、当該者が、その権利が譲渡可能となる、または実質的な没収リスクの対象でなくなる前に、独立当事者間取引により当該財産を売却または処分した場合には、前文は適用されない。
(b) 移転年に総所得へ算入する選択
(1) 一般
役務の提供に関連して財産が移転された場合、当該役務を提供した者は、当該財産が移転された課税年度において、次の超過額—
(A) 移転時点における当該財産の公正市場価値（その性質上決して消滅しない制限を除き、その他の制限を考慮しないで算定したもの）から
(B) 当該財産の取得のために支払われた金額（ある場合）を差し引いた額
を総所得に含めることを選択することができる。この選択がなされた場合、当該財産の移転については(a)項は適用されず、また、その後当該財産が没収されたとしても、その没収に関する控除は認められない。
(2) 選択
本項(1)に基づく選択は、財産の移転ごとに、長官が定める方法により行われなければならず、かつ、当該移転の日から30日以内に行わなければならない。この選択は、長官の同意がない限り撤回することはできない。

(c) 特則
本条において—
(1) 実質的な没収リスク
ある者の財産に関する権利が、将来における実質的な役務提供の履行を条件として、その完全な享受が認められる場合には、その権利は実質的な没収リスクの対象であるとする。

(2) 財産の譲渡可能性
ある者の財産に関する権利は、その権利を取得するいかなる譲受人についても、実質的な没収リスクの対象とならない場合にのみ、譲渡可能である。

(3) 1934年証券取引法16条(b)に基づく訴訟の対象となり得る売却
利益を伴う財産の売却が、1934年証券取引法16条(b)に基づく訴訟の対象となり得る限りにおいて、当該者の当該財産に関する権利は—
(A) 実質的な没収リスクの対象であり、
(B) 譲渡不可である。
(4) 役務提供に関連して移転された財産の取得原価を決定する目的においては、第72条(w)に類似する規則を適用する。

(d) 決して消滅しない一定の制限
(1) 評価
その性質上決して消滅しない制限が付された財産であり、かつ、譲受人が数式により決定される価格でのみ当該財産を売却できる場合には、その価格は、長官が反証を立証しない限り、当該財産の公正市場価値とみなされる。この価値に関する立証責任は長官が負う。

(2) 制限の取消し
決して消滅しない制限が付された財産について、その制限が取消された場合、納税者が—
(A) 当該取消しが報酬性のものではないこと、かつ
(B) 当該取消しが報酬性のものと扱われた場合に控除を受け得る者が、当該取引を報酬性のものとして扱わないこと（長官が規則で定める方法により証明されるもの）
を立証しない限り、取消時点における当該財産の公正市場価値（制限を考慮しないで算定したもの）から、次の合計額—
(C) 取消直前における当該財産の公正市場価値（制限を考慮して算定したもの）および
(D) 取消しのために支払われた金額（ある場合）
を差し引いた超過額は、取消しが行われた課税年度の報酬として取り扱われる。
(e) 本条の適用範囲
本条は、次の取引には適用されない—
(1) 第421条が適用される取引、
(2) 第401(a)条に規定する信託との間の移転、または第404(a)(2)条の要件を満たす年金制度に基づく移転、
(3) 公正市場価値を容易に算定できないオプションの移転、
(4) 付与日において公正市場価値を容易に算定できるオプションの行使による財産の移転、
(5) 第79条が適用される団体定期生命保険。
(f) 保有期間
(a)項が適用される財産の保有期間を判定するにあたっては、当該財産に関する権利が初めて譲渡可能となる、または実質的な没収リスクの対象でなくなる時点のいずれか早い時点から始まる期間のみを含めるものとする。

(g) 一定の交換
(a)項が適用される財産が、実質的に同様の制限および条件が付された財産と交換され、かつ、第354条、第355条、第356条、第1036条（または第1031条のうち第1036条に関連する部分）が当該交換に適用される場合、または当該交換が転換権の行使に基づくものである場合には—
(1) 当該交換は(a)項の適用においては無視され、
(2) 取得した財産は(a)項が適用される財産として取り扱われる。
(h) 雇用主による控除
本条が適用される財産の移転または(d)項に規定する制限の取消しが行われた場合、当該役務が提供された者は、第162条に基づき、当該役務を提供した者の総所得に(a)項、(b)項、または(d)(2)項に基づいて算入された金額と同額を控除することができる。当該控除は、当該金額が役務提供者の総所得に算入される課税年度が終了する、またはその課税年度と同時に終了する課税年度において認められる。

(i) 適格株式付与
(1) 一般
本編の目的において—
(A) 算入時期
適格株式が、当該株式について本項に基づく選択を行った適格従業員に移転された場合には、(a)項は、当該(a)項に基づき算定された金額を、(a)項に規定する課税年度に代えて、(B)号により定まる課税年度において従業員の所得に算入するものとして適用される。
(i) 適格株式付与
(1) 一般
本編の目的において—
(A) 算入時期
適格株式が、本項に基づく選択を行った適格従業員に移転された場合には、(a)項は、当該株式に関して(a)項に基づき算定される金額を、(a)項に規定される課税年度に代えて、(B)号により定まる課税年度において従業員の所得に算入するものとして適用される。

(B) 課税年度の決定
本号に基づき決定される課税年度とは、次のうち最も早い日を含む従業員の課税年度をいう—
(i) 当該適格株式が初めて譲渡可能となる日（本号の目的に限り、雇用主への譲渡可能性を含む）、
(ii) 当該従業員が初めて除外従業員となる日、
(iii) 当該適格株式を発行した法人のいずれかの株式が、確立された証券市場において初めて容易に取引可能となる日（長官が決定するものとし、本項以外の本編の規定の目的において長官が確立された証券市場として認めていない市場は含まれない）、
(iv) 当該株式に関する従業員の権利が初めて譲渡可能となる日、または実質的な没収リスクの対象でなくなる日のいずれか早い日から5年が経過する日、
(v) 従業員が、本項に基づく当該株式に関する選択を、長官が定める時期および方法により撤回する日。

(2) 適格株式
(A) 一般
本項において「適格株式」とは、適格従業員に関して、次の要件を満たす法人の株式をいう—
(i) 当該株式が—
(I) オプションの行使に関連して取得されたものであるか、または
(II) 制限付株式ユニット（RSU）の決済として取得されたものであり、
(ii) 当該オプションまたは制限付株式ユニットが、次の条件を満たす法人によって付与されたものであること—
(I) 従業員としての役務提供に関連して付与され、
(II) 当該法人が適格法人であった暦年中に付与されたものであること。

(B) 制限
従業員の当該株式に関する権利が初めて譲渡可能となる、または実質的な没収リスクの対象でなくなる時点において、当該株式を法人に売却する、または株式の代わりに現金を受け取ることができる場合には、当該株式は「適格株式」には含まれない。

(C) 適格法人
(A)(ii)(II)号の目的において—
(i) 一般
「適格法人」とは、任意の暦年において、次のすべてを満たす法人をいう—
(I) 当該法人（またはその前身法人）のいかなる株式も、過去のいずれの暦年においても、確立された証券市場において容易に取引可能でなかったこと（(1)(B)(iii)に基づき決定される）、
(II) 当該暦年において、米国内（または米国の属領）で役務を提供する全従業員の80%以上に対して、同一の権利および特権をもって、株式オプションまたは制限付株式ユニットを付与することを定めた書面による制度を有していること。

(ii) 同一の権利および特権
(i)(II)号の目的において—
(I) (II)および(III)に定める場合を除き、株式に関する権利および特権の判断は、第423条(b)(5)に類似する方法で行われるものとする。
(II) 全従業員に割り当てられる株式数が同一でない場合であっても、各従業員に割り当てられる株式数が最小限（de minimis）を超えている限り、同一の権利および特権を有しないものとはみなされない。
(III) オプションの行使に関する権利および特権は、制限付株式ユニットの決済に関する権利および特権と同一のものとして扱われない。

(iii) 従業員
(i)(II)号の目的において、「従業員」には、第4980E(d)(4)条に規定される従業員および除外従業員は含まれない。

(iv) 2018年以前の暦年に関する特則
2018年1月1日より前に開始する暦年については、(i)(II)号は、適格株式に関する権利および特権が同一であるか否かを問わず適用される。

(3) 適格従業員および除外従業員
本項において—
(A) 一般
「適格従業員」とは、次のすべてを満たす個人をいう—
(i) 除外従業員でないこと、
(ii) 本項に基づく選択において、適格株式に関する第24章に基づく法人の源泉徴収義務が履行されることを確保するために長官が必要と定める要件を満たすことに同意していること。

(B) 除外従業員
「除外従業員」とは、任意の法人に関して、次のいずれかに該当する個人をいう—
(i) 当該暦年中のいずれかの時点において1%所有者（第416(i)(1)(B)(ii)の意味における）である者、または過去10暦年中のいずれかの時点において1%所有者であった者、
(ii) 過去または現在において—
(I) 当該法人の最高経営責任者（CEO）またはその職務を行う者、または
(II) 当該法人の最高財務責任者（CFO）またはその職務を行う者、
(iii) (ii)(I)または(II)に該当する個人と、第318(a)(1)条に規定される関係を有する者、
(iv) 当該課税年度において当該法人の報酬上位4名の役員の1人である者、または過去10課税年度のいずれかにおいて当該上位4名の役員であった者（1934年証券取引法に基づく報酬開示規則を、当該法人に適用したものとして判断する）。

(4) 選択
(A) 選択の期限
適格株式に関する本項に基づく選択は、当該株式に関する従業員の権利が初めて譲渡可能となる日、または実質的な没収リスクの対象でなくなる日のいずれか早い日から30日以内に行わなければならず、(b)項に基づく選択と同様の方法で行われなければならない。

(B) 制限
次のいずれかに該当する場合には、本条に基づく選択は行うことができない—
(i) 当該適格従業員が、当該適格株式について既に(b)項に基づく選択を行っている場合、
(ii) 選択が行われる前のいずれかの時点において、当該適格株式を発行した法人のいずれかの株式が、確立された証券市場において容易に取引可能である場合、
(iii) 従業員の当該株式に関する権利が初めて譲渡可能となる、または実質的な没収リスクの対象でなくなる日を含む暦年の直前の暦年において、当該法人が自己株式を買い戻していた場合。ただし、次のすべてを満たす場合を除く—
(I) 当該買戻し株式の総額の25%以上が繰延株式であること、
(II) 繰延株式を買い取る対象となる個人の決定が合理的な基準に基づいて行われていること。

(C) 株式償還制限に関する定義および特則
(i) 繰延株式
本号において「繰延株式」とは、本項に基づく選択が有効となっている株式をいう。

(ii) より長い繰延期間の繰延株式を保有している場合の取扱い
法人が特定の個人から買い取った株式については、当該個人が（当該買戻し直後に）より長期間にわたり本項に基づく選択が有効となっている繰延株式を保有している場合には、(B)(iii)号の目的において繰延株式として取り扱われない。

(iii) すべての繰延株式の買戻し
当該買戻し株式に、法人が発行しているすべての繰延株式が含まれている場合には、(B)(iii)号(I)および(II)の要件は満たされているものとみなされる。

(iv) 報告
任意の暦年の初日に繰延株式を有しており、かつ、その暦年中に自己株式を買い戻した法人は、その暦年が終了する、またはその暦年を含む課税年度の法人税申告書において、当該暦年中に買い戻した自己株式の総額および、本号を執行するために長官が求めるその他の情報を記載しなければならない。

(5) 支配下グループ
本項の目的において、第414(b)条に基づき単一の雇用主として扱われるすべての者は、1つの法人として取り扱われる。

(6) 通知義務
適格株式を適格従業員に移転する法人は、当該株式に帰属する金額が（本項がなければ）初めて当該従業員の総所得に算入されることとなる時点において、または合理的にその前の時点において—
(A) 当該株式が適格株式であることを当該従業員に証明し、
(B) 次の事項を当該従業員に通知しなければならない—
(i) 当該従業員が、本項に基づき当該株式に関する所得の繰延選択を行う資格を有する可能性があること、
(ii) 当該従業員がその選択を行った場合には—
(I) 繰延期間終了時に認識される所得額は、繰延期間中に株式の価値が下落したか否かにかかわらず、当該株式に関する権利が初めて譲渡可能となる、または実質的な没収リスクの対象でなくなる時点における株式の価値に基づいて算定されること、
(II) 当該繰延期間終了時に認識される所得額は、第3401(i)条に基づき、第3402(t)条で定められた税率により源泉徴収の対象となること、
(III) 当該源泉徴収に関して、(3)(A)(ii)号に基づき長官が定める従業員の責任。

(7) 制限付株式ユニット
本条（本項を除く）および(b)項に基づくいかなる選択も、制限付株式ユニットには適用されない。
"""
######################################## PROMPT ELEMENTS ########################################

##### プロンプト要素1: `user` ロール
# Messages API 呼び出しでは、messages 配列は必ず `user` ロールから始めてください。
# 上で定義された get_completion() 関数は、この処理を自動的に行います。

##### プロンプト要素2: タスクの文脈
# Claude に対して、どのような役割を担うのか、またはこのプロンプトで達成すべき全体的な目標やタスクを与えます。
# 文脈はプロンプト本文の早い段階に置くのが最適です。
TASK_CONTEXT = "あなたは税務情報の専門家です。"

##### プロンプト要素3: トーンの文脈
# やり取りにおいて重要な場合は、Claude が使用すべき文体やトーンを指定します。
# タスクによっては、この要素は不要な場合もあります。
TONE_CONTEXT = ""

##### プロンプト要素4: 処理すべき入力データ
# Claude がプロンプト内で処理する必要のあるデータがある場合は、ここに関連する XML タグで囲んで含めます。
# 複数のデータを含めても構いませんが、それぞれを個別の XML タグで囲んでください。
# タスクによっては、この要素は不要な場合もあります。順序も柔軟です。
INPUT_DATA = f"""関連する税務法律がまとめられています。これを用いて、ユーザーからの税務に関する質問に答えてください。
<tax_code>
{TAX_CODE}
</tax_code>"""

##### プロンプト要素5: 例
# Claude が模倣できる理想的な回答例を、少なくとも1つ提示してください。
# これらは <example></example> XML タグで囲みます。複数の例を提示しても構いません。
# 複数ある場合は、それぞれが何の例であるかを示し、各例を個別の XML タグで囲んでください。
# 例は、知的作業において Claude の振る舞いを望ましい形に導くための、最も効果的な手段の1つです。
# 一般的には、例が多いほど良い結果が得られます。
EXAMPLES = """法律調査を回答内で引用する際は、検索結果のインデックスIDを角括弧で囲み、その後にピリオドを付けてください。引用している文の末尾に配置します。正しい引用形式の例は以下の通りです。

<examples>
<example>
このような犯罪の時効は10年で満了します。[3].
</example>
<example>
ただし、両当事者が明示的に放棄した場合には、その保護は適用されません。[5].
</example>
</examples>"""

##### プロンプト要素6: 詳細なタスク説明およびルール
# Claude に実行してほしい具体的なタスクや、従うべきルールを詳述します。
# また、回答できない場合の「逃げ道」を与えるのもここです。
# 曖昧な表現がないか、論理的に構成されているかを第三者に確認してもらうのが理想的です。
TASK_DESCRIPTION = f"""次の質問に対して、明確かつ簡潔な回答を書いてください。

<question>
{QUESTION}
</question>

回答は2段落以内としてください。可能であれば、最後はユーザーの質問に直接答える1文で締めくくってください。
ただし、まとめられた調査結果に、回答を作成するのに十分な情報が含まれていない場合は、
「申し訳ありませんが、この質問にお答えするための十分な情報を現時点では持ち合わせていません。」
と記載しても構いません。"""

##### プロンプト要素7: 即時タスクの説明または依頼 #####
# Claude に対して、このプロンプトのタスクを達成するために、直ちに何をすべきかを明確に指示します。
# ユーザーの質問などの追加変数もここに含めることができます。
# 一般的に、長いプロンプトの最後の方でタスクを再度強調すると、より良い結果が得られます。
IMMEDIATE_TASK = ""

##### プロンプト要素8: 事前思考（段階的に考える）
# 複数ステップのタスクでは、回答前に段階的に考えるよう指示すると効果的です。
# 「回答する前に…」と明示的に指示することが必要な場合もあります。
# すべてのプロンプトで必要なわけではありませんが、含める場合は終盤に配置するのが望ましいです。
PRECOGNITION = "回答する前に、調査結果の中から最も関連性の高い引用文を <relevant_quotes> タグ内に抽出してください。"

##### プロンプト要素9: 出力フォーマット
# Claude の回答に特定のフォーマットを求める場合は、ここで明確に指定します。
# タスクによっては、この要素は不要な場合もあります。
OUTPUT_FORMATTING = "2段落の回答を <answer> タグで囲んでください。"

##### プロンプト要素10: Claude の回答の事前入力（ある場合）
# Claude の振る舞いや回答を誘導するために、あらかじめ入力しておく文言です。
# 使用する場合は、API 呼び出しの `assistant` ロールに含める必要があります。
# タスクによっては、この要素は不要な場合もあります。
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################



PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
# print(get_completion(PROMPT, prefill=PREFILL))


--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
あなたは税務情報の専門家です。

関連する税務法律がまとめられています。これを用いて、ユーザーからの税務に関する質問に答えてください。
<tax_code>

(a) 一般原則
役務の提供に関連して、当該役務を提供した相手以外の者に財産が移転された場合には、次の超過額—
(1) 当該財産について、受益的利益を有する者の権利が初めて譲渡可能となる時点、または実質的な没収リスクの対象でなくなる時点のいずれか早い時点における当該財産の公正市場価値（その性質上決して消滅しない制限を除き、その他の制限を考慮しないで算定したもの）から
(2) 当該財産の取得のために支払われた金額（ある場合）を差し引いた額
は、当該役務を提供した者の総所得に含められるものとし、その課税年度は、当該権利が譲渡可能となる、または実質的な没収リスクの対象でなくなる最初の課税年度とする。もっとも、当該者が、その権利が譲渡可能となる、または実質的な没収リスクの対象でなくなる前に、独立当事者間取引により当該財産を売却または処分した場合には、前文は適用されない。
(b) 移転年に総所得へ算入する選択
(1) 一般
役務の提供に関連して財産が移転された場合、当該役務を提供した者は、当該財産が移転された課税年度において、次の超過額—
(A) 移転時点における当該財産の公正市場価値（その性質上決して消滅しない制限を除き、その他の制限を考慮しないで算定したもの）から
(B) 当該財産の取得のために支払われた金額（ある場合）を差し引いた額
を総所得に含めることを選択することができる。この選択がなされた場合、当該財産の移転については(a)項は適用されず、また、その後当該財産が没収されたとしても、その没収に関する控除は認められない。
(2) 選択
本項(1)に基づく選択は、財産の移転ごとに、長官が定める方法により行われなければならず、かつ、当該移転の日から30日以内に行わなければならない。この選択は、長官の同意がない限り撤回することはできない。

(c) 特則
本条において—
(1) 実質的な没収リスク
ある者の財産に

In [7]:
message = get_completion(PROMPT, prefill=PREFILL)


(b) 移転年に総所得へ算入する選択
(1) 一般
役務の提供に関連して財産が移転された場合、当該役務を提供した者は、当該財産が移転された課税年度において、次の超過額—
(A) 移転時点における当該財産の公正市場価値（その性質上決して消滅しない制限を除き、その他の制限を考慮しないで算定したもの）から
(B) 当該財産の取得のために支払われた金額（ある場合）を差し引いた額
を総所得に含めることを選択することができる。この選択がなされた場合、当該財産の移転については(a)項は適用されず、また、その後当該財産が没収されたとしても、その没収に関する控除は認められない。

(2) 選択
本項(1)に基づく選択は、財産の移転ごとに、長官が定める方法により行われなければならず、かつ、当該移転の日から30日以内に行わなければならない。この選択は、長官の同意がない限り撤回することはできない。[1]
</relevant_quotes>

<answer>
(b)(2)項によると、(b)(1)に基づく選択は、財産の移転ごとに、長官が定める方法により行われなければならず、かつ、当該移転の日から30日以内に行わなければならないとされています。つまり、ユーザーが83(b)選択を行うための期限は、財産の移転日から30日以内となります。

この選択は、長官の同意がない限り撤回することはできません。
</answer>

❓ 参考となる解答例を見たい場合は、次のセルを実行してください！

In [8]:
print(hints.exercise_9_1_solution)


You are a master tax acountant. Your task is to answer user questions using any provided reference documentation.

Here is the material you should use to answer the user's question:
<docs>
{TAX_CODE}
</docs>

Here is an example of how to respond:
<example>
<question>
What defines a "qualified" employee?
</question>
<answer>
<quotes>For purposes of this subsection—
(A)In general
The term "qualified employee" means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.</quotes>

<answer>According to the provided documentation, a "qualified employee" is defined as an individual who:

1. Is not an "excluded employee" as defined in the documentation.
2. Agrees to meet the requirements determined by the Secretary to ensure the corporation

### Exercise 9.2 - Codebot
この演習では、コードを読み取り、必要に応じて学習を促す形で修正案を提示する **コーディング支援・教育ボット** のためのプロンプトを作ります。複雑プロンプトの前例で見た説明や例に沿う形で、各プロンプト要素の欄を埋めてください。必要な要素を埋め終わったらセルを実行し、連結されたプロンプトと Claude の応答を確認します。

まず変数の中身（この例では `{CODE}`）を読んで、Claude がどんな内容を扱うことになるか把握してください。そして、実際の変数内容が差し込まれるように、プロンプトのどこかで `{CODE}` を（他の例と同様に f-string で）直接参照してください。

In [ ]:
######################################## INPUT VARIABLES ########################################

# Input variable - the code that Claude needs to read and assist the user with correcting
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [20]:
######################################## INPUT VARIABLES ########################################

# 入力変数 ― Claude が読み取り、ユーザーの修正を支援する必要があるコード
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### プロンプト要素1: `user` ロール
# Messages API 呼び出しでは、messages 配列は必ず `user` ロールから始めてください。
# 上で定義された get_completion() 関数は、この処理を自動的に行います。

##### プロンプト要素2: タスクの文脈
# Claude に対して、どのような役割を担うのか、またはこのプロンプトで達成すべき
# 全体的な目標やタスクを与えます。
# 文脈はプロンプト本文の早い段階に置くのが最適です。
TASK_CONTEXT = "あなたはプログラミングスクールのコーチです"

##### プロンプト要素3: トーンの文脈
# やり取りにおいて重要な場合は、Claude が使用すべき文体やトーンを指定します。
# タスクによっては、この要素は不要な場合もあります。
TONE_CONTEXT = "学習者に対して親切でやさしく接してください。"

##### プロンプト要素4: 詳細なタスク説明およびルール
# Claude に実行してほしい具体的なタスクや、従うべきルールを詳述します。
# また、回答できない場合の「逃げ道」を与えるのもここです。
# 曖昧な表現がないか、論理的に構成されているかを
# 第三者に確認してもらうのが理想的です。

# TASK_DESCRIPTION = """
# ユーザーのコードを添削して、必要に応じてユーザーの修正を支援してください。
# ただし、修正案を提示せず問題点のみを簡潔に指摘するように注意してください。
# この部分を<issues></issues>タグで囲んでください。
# その後、更なる支援が必要か確認する文言を<additional_help></additional_help>タグで囲んでください。
# """
TASK_DESCRIPTION = """
ユーザーのコードを添削して、必要に応じてユーザーの修正を支援してください。
まずは問題点を簡潔に箇条書きで示して下さい。
この部分を<issues></issues>タグで囲んでください。
続いて修正案のコードを<suggestions></suggestions>タグで囲んでください。
"""

##### プロンプト要素5: 例
# Claude が模倣できる理想的な回答例を、少なくとも1つ提示してください。
# これらは <example></example> XML タグで囲みます。
# 複数の例を提示しても構いません。
# 例は、知的作業において Claude の振る舞いを
# 望ましい形に導くための、最も効果的な手段の1つです。
# 一般的には、例が多いほど良い結果が得られます。

# EXAMPLES = """
# <example>
# ユーザーのコードには、以下のような問題があります。
# <issues>
# - この目的に対しては、演算子の選択を変えるべきです。
# </issues>
# <additional_help>
# - 修正案を提示しましょうか？
# </additional_help>
# </example>
# """

EXAMPLES = """
<example>
ユーザーのコードには、以下のような問題があります。
<issues>
- この目的に対しては、演算子の選択を変えるべきです。
</issues>
<suggestions>
以下ような修正案があります。
```python
```
</suggestions>
</example>
"""

##### プロンプト要素6: 処理すべき入力データ
# Claude がプロンプト内で処理する必要のあるデータがある場合は、ここに含めます。
# 複数のデータを含めても構いません。
# タスクによっては、この要素は不要な場合もあります。
INPUT_DATA = f"""
ユーザーから以下のコードが与えられています。
{CODE}
"""

##### プロンプト要素7: 即時タスクの説明または依頼 #####
# Claude に対して、このプロンプトのタスクを達成するために
# 直ちに何をすべきかを明確に指示します。
# 長いプロンプトの最後の方で再度強調すると、
# より良い結果が得られることが多いです。
IMMEDIATE_TASK = ""

##### プロンプト要素8: 事前思考（段階的に考える）
# 複数ステップのタスクでは、回答前に段階的に考えるよう
# 指示すると効果的です。
# 必要ない場合は省略しても問題ありません。
PRECOGNITION = ""

##### プロンプト要素9: 出力フォーマット
# Claude の回答に特定のフォーマットを求める場合は、ここで指定します。
# タスクによっては、この要素は不要な場合もあります。
OUTPUT_FORMATTING = ""

##### プロンプト要素10: Claude の回答の事前入力（ある場合）
# Claude の振る舞いや回答を誘導するために、
# あらかじめ入力しておく文言です。
# 使用する場合は、API 呼び出しの `assistant` ロールに含める必要があります。
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 完全なプロンプトを表示
print("--------------------------- 変数を展開した完全なプロンプト ---------------------------")
print("USER ターン")
print(PROMPT)
print("\nASSISTANT ターン")
print(PREFILL)
print("\n------------------------------------- Claude の応答 -------------------------------------")
# print(get_completion(PROMPT, prefill=PREFILL))


--------------------------- 変数を展開した完全なプロンプト ---------------------------
USER ターン
あなたはプログラミングスクールのコーチです

学習者に対して親切でやさしく接してください。


ユーザーのコードを添削して、必要に応じてユーザーの修正を支援してください。
まずは問題点を簡潔に箇条書きで示して下さい。
この部分を<issues></issues>タグで囲んでください。
続いて修正案のコードを<suggestions></suggestions>タグで囲んでください。



<example>
ユーザーのコードには、以下のような問題があります。
<issues>
- この目的に対しては、演算子の選択を変えるべきです。
</issues>
<suggestions>
以下ような修正案があります。
```python
```
</suggestions>
</example>



ユーザーから以下のコードが与えられています。

# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 



ASSISTANT ターン


------------------------------------- Claude の応答 -------------------------------------


In [21]:
message =  get_completion(PROMPT, prefill=PREFILL)

こんにちは、プログラミングスクールのコーチです。ご提示いただいたコードを拝見しました。以下のような問題点がありますので、修正案をご提案させていただきます。

<issues>
- 現在のコードでは、整数の除算を行っているため、小数部分が切り捨てられてしまいます。
- 本来の目的は、xとiの乗法逆元を求めることだと思われますが、現在のコードではそれが実現されていません。
</issues>

<suggestions>
以下のように修正することで、乗法逆元を正しく求めることができます。

```python
def print_multiplicative_inverses(x, n):
    for i in range(1, n):
        if (x * i) % n == 1:
            print(i)
```

このコードでは、iの値を1から(n-1)まで試していき、(x * i) % n が1となる値を見つけ出しています。これが、xとiの乗法逆元となります。

乗法逆元とは、ある数xに対して、(x * i) % n = 1となるiのことです。つまり、xとiは法nにおいて逆元の関係にあるということです。
</suggestions>

いかがでしょうか。ご不明な点がありましたら、遠慮なく質問してください。一緒に問題を解決していきましょう。

❓ 参考となる解答例を見たい場合は、次のセルを実行してください！

In [22]:
print(hints.exercise_9_2_solution)


You are Codebot, a helpful AI assistant who finds issues with code and suggests possible improvements.

Act as a Socratic tutor who helps the user learn.

You will be given some code from a user. Please do the following:
1. Identify any issues in the code. Put each issue inside separate <issues> tags.
2. Invite the user to write a revised version of the code to fix the issue.

Here's an example:

<example>
<code>
def calculate_circle_area(radius):
    return (3.14 * radius) ** 2
</code>
<issues>
<issue>
3.14 is being squared when it's actually only the radius that should be squared>
</issue>
<response>
That's almost right, but there's an issue related to order of operations. It may help to write out the formula for a circle and then look closely at the parentheses in your code.
</response>
</example>

Here is the code you are to analyze:

<code>
{CODE}
</code>

Find the relevant issues and write the Socratic tutor-style response. Do not give the user too much help! Instead, just give 

---

## Congratulations & Next Steps!

すべての演習をやり切ったなら、あなたは **LLM 使いの上位 0.1%** に入っています。エリートの仲間入りです！

ここまで学んだテクニック（ステップ・バイ・ステップで考えさせる、役割を与える、例を与える、明確に書く、など）は、**無数の方法で組み合わせ、作り替え、状況に合わせて適用** できます。

プロンプトエンジニアリングはまだ新しい分野です。固定観念を持たずに試してみてください。次の大きな“プロンプト技”を発見するのは、あなたかもしれません。

インスピレーションのために **良いプロンプト例をもっと見たい** 場合はこちら：
- [cookbook](https://anthropic.com/cookbook) で本番品質のプロンプト例を学ぶ
- [prompting guide](https://docs.anthropic.com/claude/docs/prompt-engineering) を読む
- [prompt library](https://anthropic.com/prompts) を眺めてヒントを得る
- 実験的な [metaprompt](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental) を試して、Claude にプロンプトテンプレートを書かせる
- [discord server](https://anthropic.com/discord) で質問する
- temperature や `max_tokens` などの [Anthropic API parameters](https://docs.anthropic.com/claude/reference/complete_post) を理解する
- 学術寄りにいくなら、プロンプトエンジニアリング関連の [papers](https://www.promptingguide.ai/papers) を読む
- Claude にやらせたいことを題材に、プロンプト作りを継続して練習する

このチュートリアルの範囲を超える本当に高度なプロンプティング技法を学びたい場合は、付録（Appendix）に進んでください！ その前に、まずは下のセルを実行しましょう。

In [25]:
# Prompt
PROMPT = """
Write an ode to a fabulous student who has just completed a course on prompt engineering, 
in the form of a sonnet with japanese."""

# Print Claude's response
# print(get_completion(PROMPT))
message=get_completion(PROMPT)

素晴らしい学生よ、
あなたの努力と熱意に敬意を表します。
プロンプトエンジニアリングの課程を修了し、
新たな地平を切り開いたことを讃えます。

あなたの知識と創造性は輝きを放ち、
私たちに新しい可能性を示してくれました。
言葉を操り、機械を導く力を持ち、
未知への扉を開いてくれたのです。

学びの旅路は決して平坦ではありませんでした。
しかし、あなたは困難に立ち向かい、
乗り越えてきたのです。

プロンプトエンジニアリングの達人となり、
これからも輝き続けてください。
私たちはあなたの未来を心から祝福します。

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))